In [42]:
import os
import re
import fitz
print(os.getcwd())

C:\Users\jessi\OneDrive\School\lab\Data Extraction from PDF\code\pdf-extraction\backend\module


In [43]:
pdfFile = '../data/demo/2c7d75bb5c074aff8a5e1a24c628bf39/test/公務員懲戒委員會102年度清字第11347號公懲裁定.pdf'
pdf_text = ''
if os.path.exists(pdfFile):
    print(pdfFile)
    with fitz.open(pdfFile) as pdf:
        for page in pdf:
            pdf_text = pdf_text + page.get_text("text")
else:
    print('File is not exist!')

new_pdf_text = re.sub(r'^(\s\s|)[0-9][0-9]+\n', '', pdf_text)
new_pdf_text = re.sub(r'[１-９]+\n', '', new_pdf_text)
new_pdf_text = re.sub(r'[0-9]+\n', '', new_pdf_text)
new_pdf_text = re.sub(r' ', '', new_pdf_text)
new_pdf_text = re.sub(r'^　', '', new_pdf_text)
new_pdf_text = re.sub(r'^\n', '', new_pdf_text)
# print(new_pdf_text)
pdf_text = new_pdf_text
len(pdf_text)
# print(pdf_text)

../data/demo/2c7d75bb5c074aff8a5e1a24c628bf39/test/公務員懲戒委員會102年度清字第11347號公懲裁定.pdf


627

In [44]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1500,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(pdf_text)
len(texts)

1

In [45]:
# len(texts[0])
texts[0]

'公務員懲戒委員會裁定\n\u3000\n102年度清字第11347號\n移送機關\u3000內政部\u3000\u3000設臺北市○○區○○路○號\n代\u3000表\u3000人\u3000徐國勇\u3000\u3000住同上\n被付懲戒人\u3000陳仁維\u3000\u3000臺東縣警察局警務員\n上列被付懲戒人因違法失職案件，經內政部移送審理，本會裁定\n如下：\n主文\n本件停止審議程序之議決撤銷。\n理由\n一、按公務員懲戒法中華民國104年5月1日修正之條文施行前已\n繫屬於公務員懲戒委員會之懲戒案件尚未終結者，於該法修\n正施行後，由公務員懲戒委員會合議庭依修正後之程序規定\n繼續審理。但修正施行前已依法進行之程序，其效力不受影\n響。為該法第77條第1款所明定。又停止審理程序之裁定，\n公務員懲戒委員會合議庭得依聲請或依職權撤銷之。同法第\n39條第2項定有明文。\n二、本件被付懲戒人陳仁維因違法失職案件應否受懲戒處分及處\n分之輕重，同一行為涉及刑事部分，以其犯罪是否成立為斷\n，前經本會於102年3月15日議決於刑事裁判確定前，停止審\n議程序在案。經查被付懲戒人所涉刑事案件，業經臺灣高等\n法院花蓮分院以106年度重上更�字第17號判決在案，此有\n該判決書在卷可稽。依上開說明，本會合議庭自得依職權將\n原停止審議程序之議決撤銷，繼續審理程序。\n三、依公務員懲戒法第39條第2項，裁定如主文。\n中華民國108年9月11日\n公務員懲戒委員會第二庭\n審判長委員姜仁脩\n委員吳景源\n委員洪佳濱\n委員吳謀焰\n委員蘇振堂\n以上正本證明與原本無異。\n中華民國108年9月12日\n書記官黃紋麗'

In [ ]:
# len(texts[1])
texts[1]

In [1]:
with open("API_KEY", "r", encoding="utf-8") as f:
    key = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'API_KEY'

In [47]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings(openai_api_key=key)
docsearch = FAISS.from_texts(texts, embeddings)

In [48]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.prompts import PromptTemplate

prompt_template = """使用PDF文件的上下文來回答問題，不知道答案就說不知道，或者在文件中找不到答案，請不要試圖編答案

{context}

Question: {question}
Answer in traditional Chinese:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [49]:
chain = load_qa_chain(ChatOpenAI(model_name="gpt-3.5-turbo",openai_api_key=key), chain_type="stuff", prompt=PROMPT)

In [50]:
def query_func(query):
  with get_openai_callback() as cb:
    docs = docsearch.similarity_search(query)
    res = chain.run(input_documents=docs, question=query)
    print(f"Output: {res}")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Successful Requests: {cb.successful_requests}")
    print(f"Total Cost (USD): ${cb.total_cost}")

In [14]:
query_func("請告訴我文件中的title")

Output: 公務員懲戒委員會裁定
Total Tokens: 988
Prompt Tokens: 969
Completion Tokens: 19
Successful Requests: 1
Total Cost (USD): $0.001976


In [11]:
query_func("請告訴我文件中的標題")

Output: 公務員懲戒委員會裁定
Total Tokens: 992
Prompt Tokens: 973
Completion Tokens: 19
Successful Requests: 1
Total Cost (USD): $0.001984


In [12]:
query_func("請告訴我文件中的title(標題)")

Output: 公務員懲戒委員會裁定
Total Tokens: 994
Prompt Tokens: 975
Completion Tokens: 19
Successful Requests: 1
Total Cost (USD): $0.0019879999999999997


In [13]:
query_func("title指文件中的標題，請告訴我文件中的title")

Output: 公務員懲戒委員會裁定
Total Tokens: 999
Prompt Tokens: 980
Completion Tokens: 19
Successful Requests: 1
Total Cost (USD): $0.0019979999999999998


In [20]:
query_func("請告訴我文件中的main_content")

Output: 本件停止審議程序之議決撤銷，並依公務員懲戒法第39條第2項裁定。其理由為被付懲戒人陳仁維因違法失職案件應否受懲戒處分及處分之輕重，同一行為涉及刑事部分，以其犯罪是否成立為斷，前經本會於102年3月15日議決於刑事裁判確定前，停止審議程序在案。經查被付懲戒人所涉刑事案件，業經臺灣高等法院花蓮分院以106年度重上更�字第17號判決在案，此有該判決書在卷可稽。依上開說明，本會合議庭自得依職權將原停止審議程序之議決撤銷，繼續審理程序。
Total Tokens: 1281
Prompt Tokens: 970
Completion Tokens: 311
Successful Requests: 1
Total Cost (USD): $0.002562


In [19]:
query_func("請告訴我文件中的主文")

Output: 本件停止審議程序之議決撤銷。
Total Tokens: 990
Prompt Tokens: 970
Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $0.00198


In [21]:
query_func("請告訴我文件中的main_content(主文)")


Output: 本件停止審議程序之議決撤銷。
Total Tokens: 993
Prompt Tokens: 973
Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $0.001986


In [22]:
query_func("main_content指文件中的主文，請告訴我文件中的main_content")


Output: 本件停止審議程序之議決撤銷。
Total Tokens: 999
Prompt Tokens: 979
Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $0.001998


In [23]:
query_func("請告訴我文件中的date")


Output: 中華民國108年9月11日
Total Tokens: 981
Prompt Tokens: 969
Completion Tokens: 12
Successful Requests: 1
Total Cost (USD): $0.001962


In [24]:
query_func("請告訴我文件中的日期")

Output: 中華民國108年9月11日
Total Tokens: 981
Prompt Tokens: 969
Completion Tokens: 12
Successful Requests: 1
Total Cost (USD): $0.001962


In [25]:
query_func("請告訴我文件中的date(日期)")


Output: 中華民國108年9月11日
Total Tokens: 983
Prompt Tokens: 971
Completion Tokens: 12
Successful Requests: 1
Total Cost (USD): $0.001966


In [26]:
query_func("date指文件中的日期，請告訴我文件中的date")

Output: 中華民國108年9月11日
Total Tokens: 988
Prompt Tokens: 976
Completion Tokens: 12
Successful Requests: 1
Total Cost (USD): $0.001976


In [27]:
query_func("請告訴我文件中的appeal.appellant")


Output: 文件中沒有提到appeal和appellant。
Total Tokens: 985
Prompt Tokens: 972
Completion Tokens: 13
Successful Requests: 1
Total Cost (USD): $0.00197


In [28]:
query_func("請告訴我文件中的移送機關/上訴人/聲請人/原告")

Output: 移送機關：內政部
上訴人、聲請人、原告：文件中沒有提到，無法得知。
Total Tokens: 1031
Prompt Tokens: 990
Completion Tokens: 41
Successful Requests: 1
Total Cost (USD): $0.002062


In [32]:
query_func("請告訴我文件中的移送機關")

Output: 內政部。
Total Tokens: 980
Prompt Tokens: 975
Completion Tokens: 5
Successful Requests: 1
Total Cost (USD): $0.00196


In [29]:
query_func("請告訴我文件中的appeal.appellant(移送機關/上訴人/聲請人/原告)")


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


Output: 文件中沒有提到appeal、appellant或原告等相關詞語，因此無法回答此問題。
Total Tokens: 1031
Prompt Tokens: 995
Completion Tokens: 36
Successful Requests: 1
Total Cost (USD): $0.002062


In [33]:
query_func("請告訴我文件中的appeal.appellant(移送機關)")


Output: 移送機關為內政部。
Total Tokens: 995
Prompt Tokens: 980
Completion Tokens: 15
Successful Requests: 1
Total Cost (USD): $0.00199


In [30]:
query_func("appeal.appellant指文件中的移送機關/上訴人/聲請人/原告，請告訴我文件中的appeal.appellant")

Output: 文件中的appeal.appellant指的是被付懲戒人陳仁維。
Total Tokens: 1029
Prompt Tokens: 1002
Completion Tokens: 27
Successful Requests: 1
Total Cost (USD): $0.002058


In [51]:
query_func("請告訴我文件中的代表人")

Output: 徐國勇。
Total Tokens: 978
Prompt Tokens: 971
Completion Tokens: 7
Successful Requests: 1
Total Cost (USD): $0.001956


In [55]:
query_func("請告訴我文件中的被付懲戒人")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 7843bfdd1f90d24d0a246b7891dc2242 in your message.).


Output: 被付懲戒人是陳仁維，臺東縣警察局警務員。
Total Tokens: 1014
Prompt Tokens: 977
Completion Tokens: 37
Successful Requests: 1
Total Cost (USD): $0.002028


In [35]:
query_func("請告訴我文件中的date")

Output: 中華民國108年9月11日
Total Tokens: 981
Prompt Tokens: 969
Completion Tokens: 12
Successful Requests: 1
Total Cost (USD): $0.001962


In [38]:
query_func("請告訴我文件中的理由項目有多少個?並請一一列出")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60).


Output: 文件中的理由項目有三個，分別是：
一、修正公務員懲戒法後尚未終結的案件程序規定繼續審理，且停止審理程序之裁定可依聲請或職權撤銷。
二、本件被付懲戒人所涉及刑事案件已經經臺灣高等法院判決，在判決確定前停止審議程序的議決撤銷，繼續審理程序。
三、依公務員懲戒法第39條第2項，裁定如主文。
Total Tokens: 1186
Prompt Tokens: 988
Completion Tokens: 198
Successful Requests: 1
Total Cost (USD): $0.002372


In [39]:
query_func("請一一列出文件中理由的項目")

Output: 一、公務員懲戒法修正前已繫屬於公務員懲戒委員會之懲戒案件尚未終結，修正後依程序繼續審理，但修正前已依法進行之程序效力不受影響。
二、被付懲戒人陳仁維案件涉及刑事部分，以其犯罪是否成立為斷，前經本會停止審議程序在案，現已有判決確定，故原停止審議程序之議決撤銷，繼續審理程序。
三、依公務員懲戒法第39條第2項，裁定如主文。
Total Tokens: 1204
Prompt Tokens: 973
Completion Tokens: 231
Successful Requests: 1
Total Cost (USD): $0.002408


In [41]:
query_func("請一一列出文件中理由的項目，並不做其他修改")

Output: 一、修正後程序規定繼續審理，但修正前程序效力不受影響，可撤銷停止審理程序之議決。
二、被付懲戒人所涉行為涉及刑事部分，需等待刑事裁判確定，已有該判決書在卷可稽。
三、依公務員懲戒法第39條第2項，裁定如主文。
Total Tokens: 1122
Prompt Tokens: 981
Completion Tokens: 141
Successful Requests: 1
Total Cost (USD): $0.0022440000000000003


In [56]:
query_func("請問文件中的理由")

Output: 本件停止審議程序之議決撤銷，因為被付懲戒人所涉刑事案件已經有判決，依據公務員懲戒法第77條第1款及第39條第2項，本會合議庭得依職權將原停止審議程序之議決撤銷，繼續審理程序。
Total Tokens: 1099
Prompt Tokens: 967
Completion Tokens: 132
Successful Requests: 1
Total Cost (USD): $0.002198


In [57]:
query_func("請問文件中的審判長委員")

Output: 姜仁脩
Total Tokens: 982
Prompt Tokens: 975
Completion Tokens: 7
Successful Requests: 1
Total Cost (USD): $0.001964


In [59]:
query_func("請問文件中的法庭")

Output: 公務員懲戒委員會第二庭
Total Tokens: 988
Prompt Tokens: 968
Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $0.001976


In [60]:
query_func("請問文件中的委員有哪些")

Output: 審判長委員姜仁脩、委員吳景源、委員洪佳濱、委員吳謀焰、委員蘇振堂。
Total Tokens: 1036
Prompt Tokens: 973
Completion Tokens: 63
Successful Requests: 1
Total Cost (USD): $0.002072


In [61]:
query_func("請問文件中的委員")

Output: 審判長委員姜仁脩、委員吳景源、委員洪佳濱、委員吳謀焰、委員蘇振堂。
Total Tokens: 1032
Prompt Tokens: 969
Completion Tokens: 63
Successful Requests: 1
Total Cost (USD): $0.002064


In [62]:
query_func("請問文件中的書記官")

Output: 書記官黃紋麗。
Total Tokens: 983
Prompt Tokens: 970
Completion Tokens: 13
Successful Requests: 1
Total Cost (USD): $0.001966


In [63]:
query_func("請告訴我文件中的書記官")

Output: 書記官為黃紋麗。
Total Tokens: 989
Prompt Tokens: 973
Completion Tokens: 16
Successful Requests: 1
Total Cost (USD): $0.001978
